# Gather Spotify Data
### Use Spotify API to get playlist and track data. Save it to a file to be used in other notebooks. No need to rerun this code (takes several hours).

In [31]:
# Import libraries
import requests 
import sys
import spotipy
import spotipy.util as util
import os
import json
import sys
import time
import pickle
import string
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

#### Local file directory (update this with your own directory)

In [9]:
# Local data folder to save data to and pull data from
### NOTE to partners: comment out mine; use your own local folder path
### ^^ you'll have to continutally check this with each push/pull from GitHub ^^

# Daniel
data_folder = "/Users/dannynightingale/Documents/Harvard 2017-2018/Harvard 2017-2018 Fall/CS 109a/Spotify Final Project/data/"

# Hartin
# data_folder = "/Users/hartincode/Documents/Harvard/Senior_Fall/CS109/final_project/Spotify Data"

# Will
# data_folder = REPLACE


In [3]:
# set up required authorization function --> run this every time we want to query Spotify for data
# NOTE: use auth()[0] for spotipy function and auth()[1] for json request function
def auth():
    os.environ["SPOTIPY_CLIENT_ID"] = '574cd0d54f654384a8b060c01844e82b'
    os.environ["SPOTIPY_CLIENT_SECRET"] = '299e1821f0864ff79bbeaec6fd2fc4d8'
    os.environ["SPOTIPY_REDIRECT_URI"] = 'http://localhost/'

    scope = 'user-library-read'
    if len(sys.argv) > 1:
        username = sys.argv[1]
    else:
        print("Usage: %s username" % (sys.argv[0],))
        sys.exit()

    token = util.prompt_for_user_token(username, scope)
    
    sp = spotipy.Spotify(auth=token) # spotipy function setup
    request_token = "Bearer " + token # json request function setup
    
    return(sp, request_token)

In [214]:
# Get the category IDS and names
get_cat = auth()[0].categories(country=None, locale=None, limit=50, offset=0)
categories = get_cat['categories']['items']  
cat_ids = []
cat_names = []

for category in categories:
    cat_ids.append(category['id'])
for category in categories:
    cat_names.append(category['name'])

print("Category Names:")
for i in cat_names:
    print(" ", i)

Category Names:
  Top Lists
  Chill
  Mood
  Pop
  Electronic/Dance
  Hip-Hop
  Party
  Rock
  Workout
  Focus
  Decades
  Dinner
  Sleep
  Indie
  R&B
  Trending
  Metal
  Soul
  Romance
  Jazz
  Classical
  Latin
  Country
  Folk & Americana
  Blues
  Travel
  Kids
  Reggae
  Gaming
  Punk
  Funk
  Comedy


### Do not rerun. Takes a while to get a list of playlists

In [ ]:
###### Get a list of playlists (DO NOT RERUN --> load from file)

# array for the playlist hrefs
playlist_hrefs = []

### do searches for for different search words --> get a whole variety of playlists
# search for the main category names plus a few others

searches = cat_names[:]
others = ['music', 'playlist', 'holiday', 'classic rock', 'popular', 'kpop', 'dance', 'twee', 'rap', \
        'oldies', 'world', 'christian', 'gospel', 'mix', 'Taylor', 'Ed Sheeran', 'Imagine Dragons', \
        'Maroon 5', 'Adele', 'Nicki', 'Beyonce', 'Rihanna']
for query in others:
    searches.append(query)

print("Searches:", searches, "\n")

# search each category / search word for playlists
for search in searches:
    search_request = "https://api.spotify.com/v1/search?query=" + search + "&offset=0&limit=50&type=playlist"
    search_playlists = requests.get("https://api.spotify.com/v1/search?query=music&offset=0&limit=50&type=playlist", \
                                    headers={"Authorization": auth()[1]})
    playlists = json.loads(search_playlists.text)["playlists"]

    
    # number of playlists counter (count loops)
    counter = 1
    
    ### loads 50 at a time --> loop through and get all of the playlist hrefs
    while search_playlists:

        time.sleep(.02)
        sys.stdout.write("\r{}: {} playlists".format(search, counter*50))
        sys.stdout.flush()
        counter = counter + 1

        for item in playlists['items']:
            playlist_hrefs.append(item['href'])

        if playlists['next']:   

            try: 
                search_playlists = requests.get(playlists['next'], headers={"Authorization": auth()[1]})  
            except: # Sometimes exceptions from server --> try again                    
                search_playlists = requests.get(playlists['next'], headers={"Authorization": auth()[1]})

            # if good --> set the returned query as the playlist output    
            if search_playlists.status_code == 200:
                    playlists = json.loads(search_playlists.text)["playlists"]
            else: # Sometimes get bad response --> try again
                time.sleep(2)
                search_playlists = requests.get(playlists['next'], headers={"Authorization": auth()[1]})
                
                # if good --> set the returned query as the playlist output
                if search_playlists.status_code == 200:
                    playlists = json.loads(search_playlists.text)["playlists"]
                
                else: # If still bad, quit this loop --> means that it is time to move on to next search
                    time.sleep(.5)
                    print ("\n")
                    continue
        else: # if no more, done with while loop
            search_playlists = None


In [ ]:
# unique playlists 
playlist_hrefs_unique = np.unique(playlist_hrefs)

In [ ]:
# ###### DO NOT RERUN; save playlist hrefs for future use (so that do not need to repeat)
# with open(data_folder + "playlist_hrefs_unique.txt", "wb") as fp:   #Pickling
#     pickle.dump(playlist_hrefs_unique, fp)

# with open(data_folder + "playlist_hrefs_all.txt", "wb") as fp:
#     pickle.dump(playlist_hrefs, fp)

### Load the playlists that we want to use:

In [7]:
# Load / retrieve list of playlist hrefs (just load the list of unique hrefs)
with open(data_folder + "playlist_hrefs_unique.txt", "rb") as fp:   # Unpickling
    playlist_hrefs_loaded = pickle.load(fp)

# DO NOT RERUN THIS SECTION
### Takes hours to run

In [12]:
### Get track and playlist data --> store in arrays (DO NOT RERUN)

# create arrays for all of the song and playlist data that we want
pl_href = []
pl_id = []
pl_name = []
pl_description = []
followers = []
pl_length = []

track_album_name = []
track_album_id = []
track_artists_name1 = []
track_artists_id1 = []
track_artists_name2 = []
track_artists_id2 = []
track_artists_name3 = []
track_artists_id3 = []
track_artists_name4 = []
track_artists_id4 = []
track_artists_name5 = []
track_artists_id5 = []
track_href = []
track_id = []
track_populatrity = []
track_length = []
track_name = []
track_explicit = []

track_added = []
track_isrc = []

# track errors and start progress counter
error_href = []
counter = 1

# loop through list of playlists
for playlist_href in playlist_hrefs_loaded[14:]:
    
    # write progress 
    sys.stdout.write("\rplaylist {}".format(counter))
    sys.stdout.flush()
    counter = counter + 1
    
    # get the playlist information
    try: 
        request = requests.get(playlist_href, headers={"Authorization": auth()[1]})
    except: # Sometimes exceptions from server --> try again                    
        request = requests.get(playlist_href, headers={"Authorization": auth()[1]})

    # if good --> set the returned query as the playlist output    
    if request.status_code == 200:
            playlist = json.loads(request.text)
    else: # Sometimes get bad response --> try again
        time.sleep(2)
        request = requests.get(playlist_href, headers={"Authorization": auth()[1]})

        # if good --> set the returned query as the playlist output
        if request.status_code == 200:
            playlist = json.loads(request.text)

        else: # If still bad, quit this loop --> means that it is time to move on to next playlist
            time.sleep(.1)
            print ("\n")
            error_href.append(playlist_href) # Add error href to error array (maybe to back to this playlist)
            continue
    
    # prepare to get track data
    tracks = playlist['tracks']['items']
    tracks_href = playlist['tracks']['href']
    tracks_next = playlist['tracks']['next']

    # loop through multiple sets of 100 tracks (maximum in one href)
    while tracks != None:
        
        # get track info and playlist info (for each track within playlist)
        for track in tracks:

            # track info variable
            info = track['track']

            # test to see if loaded correctly --> if not, continue
            try: 
                track_album_name.append(info['album']['name'])
                track_album_id.append(info['album']['id'])
            except: 
                continue
            
            # get playlist info
            pl_href.append(playlist['href'])
            pl_id.append(playlist['id'])
            pl_name.append(playlist['name'])
            pl_description.append(playlist['description'])
            followers.append(playlist['followers']['total'])
            pl_length.append(playlist['tracks']['total'])
            
                
            # get the track's first five artists
            artist_names = []
            artist_ids = []

            for artist in info['artists']:
                artist_names.append(artist['name'])
                artist_ids.append(artist['id'])

            # depending on the number of artists, fill in these arrays differently
            if(len(artist_ids)==1): # 1 artist
                track_artists_name1.append(artist_names[0])
                track_artists_id1.append(artist_ids[0])
                track_artists_name2.append("NA")
                track_artists_id2.append("NA")
                track_artists_name3.append("NA")
                track_artists_id3.append("NA")
                track_artists_name4.append("NA")
                track_artists_id4.append("NA")
                track_artists_name5.append("NA")
                track_artists_id5.append("NA")
            elif(len(artist_ids)==2): # 2 artists
                track_artists_name1.append(artist_names[0])
                track_artists_id1.append(artist_ids[0])
                track_artists_name2.append(artist_names[1])
                track_artists_id2.append(artist_ids[1])
                track_artists_name3.append("NA")
                track_artists_id3.append("NA")
                track_artists_name4.append("NA")
                track_artists_id4.append("NA")
                track_artists_name5.append("NA")
                track_artists_id5.append("NA")
            elif(len(artist_ids)==3): # 3 artists
                track_artists_name1.append(artist_names[0])
                track_artists_id1.append(artist_ids[0])
                track_artists_name2.append(artist_names[1])
                track_artists_id2.append(artist_ids[1])
                track_artists_name3.append(artist_names[2])
                track_artists_id3.append(artist_ids[2])
                track_artists_name4.append("NA")
                track_artists_id4.append("NA")
                track_artists_name5.append("NA")
                track_artists_id5.append("NA")
            elif(len(artist_ids)==4): # 4 artists
                track_artists_name1.append(artist_names[0])
                track_artists_id1.append(artist_ids[0])
                track_artists_name2.append(artist_names[1])
                track_artists_id2.append(artist_ids[1])
                track_artists_name3.append(artist_names[2])
                track_artists_id3.append(artist_ids[2])
                track_artists_name4.append(artist_names[3])
                track_artists_id4.append(artist_ids[3])
                track_artists_name5.append("NA")
                track_artists_id5.append("NA")
            else: # 5 or more artists
                track_artists_name1.append(artist_names[0])
                track_artists_id1.append(artist_ids[0])
                track_artists_name2.append(artist_names[1])
                track_artists_id2.append(artist_ids[1])
                track_artists_name3.append(artist_names[2])
                track_artists_id3.append(artist_ids[2])
                track_artists_name4.append(artist_names[3])
                track_artists_id4.append(artist_ids[3])
                track_artists_name5.append(artist_names[4])
                track_artists_id5.append(artist_ids[4])

            # get other track info
            track_href.append(info['href'])
            track_id.append(info['id'])
            track_populatrity.append(info['popularity'])
            track_length.append(info['duration_ms'])
            track_name.append(info['name'])
            track_explicit.append(info['explicit'])
            track_added.append(track['added_at'])
            try: # a few of them are missing this field --> try though
                track_isrc.append(info['external_ids']['isrc'])
            except:
                track_isrc.append("NA")

        # get next set of tracks for playlist
        try: 
            request2 = requests.get(tracks_next, headers={"Authorization": auth()[1]})
        except: # If error, continue                
            tracks = None
            continue
            
        # if good --> set the returned query as the track output    
        if request2.status_code == 200:
                tracks = json.loads(request2.text)['items']
                tracks_next = json.loads(request2.text)['next']
        else: # Sometimes get bad response --> try again
            request2 = requests.get(tracks_next, headers={"Authorization": auth()[1]})

            # if good --> set the returned query as the track output
            if request2.status_code == 200:
                tracks = json.loads(request2.text)['items']
                tracks_next = json.loads(request2.text)['next']

            else: # If still bad, quit this loop --> means that it is time to move on to next playlist
                tracks = None
                continue

playlist 3794

playlist 9012

playlist 10154

In [220]:
### Create the dictionary for the playlists and tracks (DO NOT RERUN)

# Stack the values
stacked_values_lists = [pl_href, pl_id, pl_name, pl_description, followers, pl_length, track_album_name, track_album_id, \
                        track_artists_name1, track_artists_id1, track_artists_name2, track_artists_id2, \
                        track_artists_name3, track_artists_id3, track_artists_name4, track_artists_id4, \
                        track_artists_name5, track_artists_id5, track_href, track_id, track_populatrity, \
                        track_length, track_name, track_explicit, track_added, track_isrc]
# print(stacked_values_lists)

# Zip values together
stacked_values_iterator = zip(*stacked_values_lists)

# name the column headers (or dictionary keys)
column_headers = ['pl_href', 'pl_id', 'pl_name', 'pl_description', 'followers', 'pl_length', 'track_album_name', 'track_album_id', \
                        'track_artists_name1', 'track_artists_id1', 'track_artists_name2', 'track_artists_id2', \
                        'track_artists_name3', 'track_artists_id3', 'track_artists_name4', 'track_artists_id4', \
                        'track_artists_name5', 'track_artists_id5', 'track_href', 'track_id', 'track_populatrity', \
                        'track_length_ms', 'track_name', 'track_explicit', 'track_added', 'track_isrc']

# Create the dictionary
tracks_dict = [{col: val for col, val in zip(column_headers, col_values)} for col_values in stacked_values_iterator]

# Obtained the expected dictionary?
tracks_dict[500]


{'followers': 4060,
 'pl_description': 'Most of the songs from Now thats what I call Music 50 to the most recent Now album. Giving a range of songs, perfect when you are not sure what to listen to.',
 'pl_href': 'https://api.spotify.com/v1/users/08vlaker/playlists/1fjd8SfX9TksiG6IKViCg5',
 'pl_id': '1fjd8SfX9TksiG6IKViCg5',
 'pl_length': 1925,
 'pl_name': 'Now 50 to 98',
 'track_added': '2015-04-29T07:08:53Z',
 'track_album_id': '5uR0dqBMYWZFYJT7mvPZ82',
 'track_album_name': 'The Lady Killer (Deluxe)',
 'track_artists_id1': '5nLYd9ST4Cnwy6NHaCxbj8',
 'track_artists_id2': 'NA',
 'track_artists_id3': 'NA',
 'track_artists_id4': 'NA',
 'track_artists_id5': 'NA',
 'track_artists_name1': 'CeeLo Green',
 'track_artists_name2': 'NA',
 'track_artists_name3': 'NA',
 'track_artists_name4': 'NA',
 'track_artists_name5': 'NA',
 'track_explicit': False,
 'track_href': 'https://api.spotify.com/v1/tracks/7kPYMrNaz5UTih75OfjTIV',
 'track_id': '7kPYMrNaz5UTih75OfjTIV',
 'track_isrc': 'USAT21002307',
 '

In [38]:
# # DO NOT RERUN THIS CELL (Store playlist/track dictionary in pickle file)

# with open(data_folder + 'tracks_dict.pickle', 'wb') as handle:
#     pickle.dump(tracks_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [119]:
# ### Get the song audio features (DO NOT RERUN)

# create arrays
track_id_unique = []

track_acousticness = []
track_danceability = []
track_energy = []
track_instrumentalness = []
track_key = []
track_liveness = []
track_loudness = []
track_mode = []
track_speechiness = []
track_tempo = []
track_time_signature = []
track_valence = []

track_audio_error = []
counter2 = 1

# get unique list of track_ids
unique_tracks = set(track_id)
unique_tracks = list(unique_tracks)

print("Total songs:", len(unique_tracks), "\n")


from itertools import zip_longest
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

grouped_tracks = grouper(unique_tracks, 50, fillvalue='')


for i in grouped_tracks:
    try:
        audios = auth()[0].audio_features(i)
        for index, audio in enumerate(audios):
            track_acousticness.append(audio['acousticness'])
            track_danceability.append(audio['danceability'])
            track_energy.append(audio['energy'])
            track_instrumentalness.append(audio['instrumentalness'])
            track_key.append(audio['key'])
            track_liveness.append(audio['liveness'])
            track_loudness.append(audio['loudness'])
            track_mode.append(audio['mode'])
            track_speechiness.append(audio['speechiness'])
            track_tempo.append(audio['tempo'])
            track_time_signature.append(audio['time_signature'])
            track_valence.append(audio['valence'])
            track_id_unique.append(i[index])
            
            sys.stdout.write("\rplaylist {}".format(counter2))
            sys.stdout.flush()
            counter2 = counter2 + 1
            
    except:
        for value in i:
            track_audio_error.append(value)           
            

Total songs: 1144412 

playlist 1135951

In [ ]:
# try error audio tracks again (DO NOT RERUN)
counter3 = 1

print("Total error tracks:", len(track_audio_error))

# loop through each track in the array of tracks that could not get audio info for
for track in track_audio_error:
    
    # write progress 
    sys.stdout.write("\rsong {}".format(counter3))
    sys.stdout.flush()
    counter3 = counter3 + 1
     
    # get audio features
    try:
        audio = auth()[0].audio_features(track)[0]
        track_acousticness.append(audio['acousticness'])
        track_danceability.append(audio['danceability'])
        track_energy.append(audio['energy'])
        track_instrumentalness.append(audio['instrumentalness'])
        track_key.append(audio['key'])
        track_liveness.append(audio['liveness'])
        track_loudness.append(audio['loudness'])
        track_mode.append(audio['mode'])
        track_speechiness.append(audio['speechiness'])
        track_tempo.append(audio['tempo'])
        track_time_signature.append(audio['time_signature'])
        track_valence.append(audio['valence'])
        track_id_unique.append(track)
    except:
        try: # Try one more time
            audio = auth()[0].audio_features(track)[0]
            track_acousticness.append(audio['acousticness'])
            track_danceability.append(audio['danceability'])
            track_energy.append(audio['energy'])
            track_instrumentalness.append(audio['instrumentalness'])
            track_key.append(audio['key'])
            track_liveness.append(audio['liveness'])
            track_loudness.append(audio['loudness'])
            track_mode.append(audio['mode'])
            track_speechiness.append(audio['speechiness'])
            track_tempo.append(audio['tempo'])
            track_time_signature.append(audio['time_signature'])
            track_valence.append(audio['valence'])
            track_id_unique.append(track)
            
        except: # If error again, add NA for that track
            track_acousticness.append("NA")
            track_danceability.append("NA")
            track_energy.append("NA")
            track_instrumentalness.append("NA")
            track_key.append("NA")
            track_liveness.append("NA")
            track_loudness.append("NA")
            track_mode.append("NA")
            track_speechiness.append("NA")
            track_tempo.append("NA")
            track_time_signature.append("NA")
            track_valence.append("NA")
            track_id_unique.append(track)

In [189]:
### Create the dictionary for the track audiofeatures (DO NOT RERUN)

# Stack the values
stacked_values_lists2 = [track_id_unique, track_acousticness, track_danceability, \
                        track_energy, track_instrumentalness, track_key, \
                        track_liveness, track_loudness, track_mode, track_speechiness, \
                        track_tempo, track_time_signature, track_valence]

# Zip values together
stacked_values_iterator2 = zip(*stacked_values_lists2)

# name the column headers (or dictionary keys)
column_headers2 = ['track_id', 'track_acousticness', 'track_danceability', \
                        'track_energy', 'track_instrumentalness', 'track_key', \
                        'track_liveness', 'track_loudness', 'track_mode', 'track_speechiness', \
                        'track_tempo', 'track_time_signature', 'track_valence']

# Create the dictionary
audio_dict = [{col: val for col, val in zip(column_headers2, col_values)} for col_values in stacked_values_iterator2]

# Obtained the expected dictionary?
audio_dict[0]

{'track_acousticness': 0.501,
 'track_danceability': 0.813,
 'track_energy': 0.476,
 'track_id': '0FKSzDPOsmrIe7VwRO1KTk',
 'track_instrumentalness': 0.00492,
 'track_key': 2,
 'track_liveness': 0.187,
 'track_loudness': -11.808,
 'track_mode': 0,
 'track_speechiness': 0.267,
 'track_tempo': 99.987,
 'track_time_signature': 4,
 'track_valence': 0.262}

In [145]:
# # DO NOT RERUN THIS CELL (Store audio dictionary in pickle file)
# with open(data_folder + 'audio_dict.pickle', 'wb') as handle:
#     pickle.dump(audio_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# get list of all of the artists in the data
track_artists = np.append(track_artists_id1, [track_artists_id2, track_artists_id3, track_artists_id4, \
                         track_artists_id5])
unique_artists = set(track_artists)
unique_artists = list(unique_artists)

In [111]:
# get artist genres (and popularity)
artist_genre = []
artist_id_unique = []
artist_popularity = []

artist_error = []
counter4 = 1

print("Total Artists:", len(unique_artists), "\n")


from itertools import zip_longest
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

grouped_artists = grouper(unique_artists, 50, fillvalue='')

for i in grouped_artists:
    try:
        artists = auth()[0].artists(i)['artists']
        for index, artist in enumerate(artists):
            artist_popularity.append(artist['popularity'])
            artist_genre.append(artist['genres'])
            artist_id_unique.append(i[index])
            
            sys.stdout.write("\rartist {}".format(counter4))
            sys.stdout.flush()
            counter4 = counter4 + 1        
    except:
        artist_error.append(i)         

Total Artists: 211429 

playlist 211255

In [114]:
# for errors, try one more time
for i in artist_error:
    try:
        artists = auth()[0].artists(i)['artists']
        for index, artist in enumerate(artists):
            artist_popularity.append(artist['popularity'])
            artist_genre.append(artist['genres'])
            artist_id_unique.append(i[index])
                
    except:
        continue        

In [ ]:
### Create the dictionary for the artist info (DO NOT RERUN)

# Stack the values
stacked_values_lists3 = [artist_id_unique, artist_genre, artist_popularity]

# Zip values together
stacked_values_iterator3 = zip(*stacked_values_lists3)

# name the column headers (or dictionary keys)
column_headers3 = ['artist_id', 'artist_genre', 'artist_popularity']

# Create the dictionary
artist_dict = [{col: val for col, val in zip(column_headers3, col_values)} for col_values in stacked_values_iterator3]

# Obtained the expected dictionary?
artist_dict[:10]

In [121]:
# # DO NOT RERUN THIS CELL (Store artist dictionary in pickle file)
# with open(data_folder + 'artist_dict.pickle', 'wb') as handle:
#     pickle.dump(artist_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [142]:
# get list of artist genres
artist_genres_unique = list(itertools.chain.from_iterable(artist_genre))
artist_genres_unique = set(artist_genres_unique)
artist_genres_unique = list(artist_genres_unique)

In [243]:
# ###### DO NOT RERUN; save list of artist genres
# with open(data_folder + "artist_genres_unique.txt", "wb") as fp:   #Pickling
#     pickle.dump(artist_genres_unique, fp)

In [6]:
with open(data_folder + 'artist_genres_unique.txt', 'rb') as handle2:
    artist_genres_unique = pickle.load(handle2)

In [28]:
# print a few genres
for i in artist_genres_unique[0:10]:
    print(i)

dub
mexican rock-and-roll
electronicore
reggae fusion
cante flamenco
post-post-hardcore
marching band
noise rock
deep free jazz
experimental dubstep


### Below here is where we load the dictionaries from pickle files:
### These can be rerun

In [10]:
# RERUN (Load playlist dictionary from pickle file)

# Playlist and track
with open(data_folder + 'tracks_dict.pickle', 'rb') as handle1:
    tracks_dictionary = pickle.load(handle1)

In [11]:
# RERUN (Load audio info dictionary from pickle file)

# Audio features
with open(data_folder + 'audio_dict.pickle', 'rb') as handle2:
    audio_dictionary = pickle.load(handle2)

In [12]:
# RERUN (Load artist info dictionary from pickle file)

# Artist features
with open(data_folder + 'artist_dict.pickle', 'rb') as handle2:
    artist_dictionary = pickle.load(handle2)

In [13]:
# Create pandas playlist and track dataframe
data_tracks = pd.DataFrame.from_dict(tracks_dictionary)

In [14]:
data_tracks[300:].head(3)

,followers,pl_description,pl_href,pl_id,pl_length,pl_name,track_added,track_album_id,track_album_name,track_artists_id1,...,track_artists_name3,track_artists_name4,track_artists_name5,track_explicit,track_href,track_id,track_isrc,track_length_ms,track_name,track_populatrity
300,4060,Most of the songs from Now thats what I call M...,https://api.spotify.com/v1/users/08vlaker/play...,1fjd8SfX9TksiG6IKViCg5,1925,Now 50 to 98,2015-04-29T06:54:42Z,6sAeDMYVsr4YUFA8aWk4yj,Fortune (Deluxe Version),7bXgB6jMjp9ATFy66eO08Z,...,NA,NA,NA,False,https://api.spotify.com/v1/tracks/12Ns5IphkblP...,12Ns5IphkblPmHxpRILG9t,USRC11200464,222306,Don't Wake Me Up,63
301,4060,Most of the songs from Now thats what I call M...,https://api.spotify.com/v1/users/08vlaker/play...,1fjd8SfX9TksiG6IKViCg5,1925,Now 50 to 98,2015-04-29T06:54:42Z,3uncLtCxOOC8UAeMvd8jey,You Bring Me Joy,6C5q61menaSerLWhltM58b,...,NA,NA,NA,False,https://api.spotify.com/v1/tracks/01iy1rlxkZyq...,01iy1rlxkZyq1adDT4I8iv,GBHMU1200182,231533,You Bring Me Joy,40
302,4060,Most of the songs from Now thats what I call M...,https://api.spotify.com/v1/users/08vlaker/play...,1fjd8SfX9TksiG6IKViCg5,1925,Now 50 to 98,2015-04-29T06:54:42Z,5L3HMFdqurRbTdhe6yOKKO,Taking Over Me,2Ah7S4oLgEBYRbz2V0cZzU,...,NA,NA,NA,False,https://api.spotify.com/v1/tracks/29JRRWetPcFX...,29JRRWetPcFXeRIClIxrsq,GBUM71204347,190215,Taking Over Me,25


In [15]:
# Create track and audio info dataframe
data_audio = pd.DataFrame.from_dict(audio_dictionary)
data_audio.drop_duplicates(subset='track_id', keep='first', inplace=True)

In [16]:
data_audio.head()

,track_acousticness,track_danceability,track_energy,track_id,track_instrumentalness,track_key,track_liveness,track_loudness,track_mode,track_speechiness,track_tempo,track_time_signature,track_valence
0,0.501,0.813,0.476,0FKSzDPOsmrIe7VwRO1KTk,0.00492,2,0.187,-11.808,0,0.267,99.987,4,0.262
1,0.796,0.569,0.517,4JiauMMJWrrz1dbl43sJRj,0.9,0,0.109,-8.647,0,0.0358,134.981,4,0.63
2,0.953,0.106,0.0721,0514vhH4KBZmNHHVZWfmXD,0.499,8,0.193,-29.75,0,0.0431,91.059,4,0.0353
3,0.0254,0.78,0.482,2ZNcAjINatNtr3ZwQaTLWd,0,2,0.104,-8.713,1,0.0422,133.992,4,0.374
4,0.627,0.548,0.204,41yOnpEZ78XRbqTWpE1ccU,0,5,0.287,-15.19,1,0.0242,99.245,4,0.0634


In [17]:
# Create artist info dataframe
data_artist = pd.DataFrame.from_dict(artist_dictionary)
data_artist.drop_duplicates(subset='artist_id', keep='first', inplace=True)

In [18]:
data_artist.head()

,artist_genre,artist_id,artist_popularity
0,[],2FaTVFX0KAyHQd0olLG0RB,1
1,[deep chill],76ZrJ8XwJXmTKbR7KDL1KW,27
2,[],3w7GgQaRMTCjaIfKRI07gN,7
3,[deep taiwanese pop],2nYN3WH8bZSMhNRGYfBT3Y,33
4,[],44SQjq9FW4yPRpMxGIO0Em,28


In [19]:
# merge the dataframes by track ID and artist ID (and sort by followers)
spotify_data = data_tracks.merge(data_audio, left_on="track_id", right_on="track_id")
spotify_data = spotify_data.merge(data_artist, left_on="track_artists_id1", right_on="artist_id", how='left')
spotify_data.rename(index=str, columns={"artist_popularity": "artist_popularity1", "artist_genre" : "artist_genre1",
                        }, inplace=True)
spotify_data = spotify_data.merge(data_artist, left_on="track_artists_id2", right_on="artist_id", how='left')
spotify_data.rename(index=str, columns={"artist_popularity": "artist_popularity2", "artist_genre" : "artist_genre2",
                        }, inplace=True)
spotify_data = spotify_data.merge(data_artist, left_on="track_artists_id3", right_on="artist_id", how='left')
spotify_data.rename(index=str, columns={"artist_popularity": "artist_popularity3", "artist_genre" : "artist_genre3",
                        }, inplace=True)
spotify_data = spotify_data.merge(data_artist, left_on="track_artists_id4", right_on="artist_id", how='left')
spotify_data.rename(index=str, columns={"artist_popularity": "artist_popularity4", "artist_genre" : "artist_genre4",
                        }, inplace=True)
spotify_data = spotify_data.merge(data_artist, left_on="track_artists_id5", right_on="artist_id", how='left')
spotify_data.rename(index=str, columns={"artist_popularity": "artist_popularity5", "artist_genre" : "artist_genre5",
                        }, inplace=True)
spotify_data.sort_values(by='followers', ascending=False, inplace=True)


In [20]:
# replace NA and blank values with pandas Nan values
spotify_data = spotify_data.replace('NA',np.NaN)
spotify_data = spotify_data.replace('',np.NaN)
spotify_data = spotify_data.replace('[]',np.NaN)

In [21]:
spotify_data.shape

(2616111, 53)

In [22]:
# put the genres for all of the artists in one column
spotify_data['artist_genre1'] = spotify_data['artist_genre1'].fillna('')
spotify_data['artist_genre2'] = spotify_data['artist_genre2'].fillna('')
spotify_data['artist_genre3'] = spotify_data['artist_genre3'].fillna('')
spotify_data['artist_genre4'] = spotify_data['artist_genre4'].fillna('')
spotify_data['artist_genre5'] = spotify_data['artist_genre5'].fillna('')

spotify_data['track_artist_genres'] = spotify_data["artist_genre1"].map(str) + spotify_data["artist_genre2"].map(str) \
+ spotify_data["artist_genre3"].map(str) + spotify_data["artist_genre4"].map(str) + spotify_data["artist_genre5"].map(str)

In [23]:
# calculate the average artist popularity
spotify_data['track_artist_popularity'] = spotify_data[['artist_popularity1', 'artist_popularity2', 'artist_popularity3', \
    'artist_popularity4', 'artist_popularity5']].mean(axis=1)

In [24]:
# delete merged on artist IDs
del spotify_data['artist_id']
del spotify_data['artist_id_x']
del spotify_data['artist_id_y']

In [25]:
# delete other individual artist characteristics
other_col_delete = spotify_data.columns[38:-2]
for i in other_col_delete:
    del spotify_data[i]

In [26]:
spotify_data.head()

,followers,pl_description,pl_href,pl_id,pl_length,pl_name,track_added,track_album_id,track_album_name,track_artists_id1,...,track_key,track_liveness,track_loudness,track_mode,track_speechiness,track_tempo,track_time_signature,track_valence,track_artist_genres,track_artist_popularity
1559039,4594125,"""mint is original, Spotify's freshest electron...",https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,60,mint,2017-11-17T02:38:57Z,3MayKx4qydJTtqesdPJz3t,Off The Deep End Volume One,5ttgIeUVka6FLyi00Uu5h8,...,5.0,0.1120,-5.178,0.0,0.0652,180.153,4.0,0.398,"['bass trap', 'big room', 'brostep', 'catstep'...",62.500000
893872,4594125,"""mint is original, Spotify's freshest electron...",https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,60,mint,2017-11-17T02:38:57Z,0t9xxSsMcwpf9AVcH1o9q9,Without You (Remixes),1vCWHaC5f2uS3yhpwWbIA6,...,2.0,0.0741,-4.851,1.0,0.0809,131.094,4.0,0.470,"['big room', 'dance pop', 'edm', 'pop', 'tropi...",79.666667
334899,4594125,"""mint is original, Spotify's freshest electron...",https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,60,mint,2017-11-19T15:45:15Z,23nYYartVhPNNmEdCeyS2x,Message In A Bottle,7AbYSzOocUvzy7DSJ9gm68,...,1.0,0.0818,-3.936,0.0,0.1030,120.076,4.0,0.370,[][],44.500000
65251,4594125,"""mint is original, Spotify's freshest electron...",https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,60,mint,2017-11-17T02:38:57Z,4iCNvVip0iPumdWvnDXrlw,Almost Home,14Tg9FvbNismPR1PJHxRau,...,0.0,0.0970,-4.873,0.0,0.0453,120.026,4.0,0.674,"['big room', 'deep big room', 'deep tropical h...",64.000000
1198752,4594125,"""mint is original, Spotify's freshest electron...",https://api.spotify.com/v1/users/spotify/playl...,37i9dQZF1DX4dyzvuaRJ0n,60,mint,2017-11-17T02:38:57Z,1GBB9KOZlpCKeajYypiJ0C,Nobody Compares To You,2ZRQcIgzPCVaT9XKhXZIzh,...,2.0,0.2360,-4.705,1.0,0.0420,104.921,4.0,0.369,"['big room', 'deep tropical house', 'edm', 'el...",73.000000


In [241]:
# percent "missingness" for each variable (only inaccurate for genres)
(spotify_data.isnull().sum()/spotify_data.shape[0])*100

followers                   0.000000
pl_description             49.556804
pl_href                     0.000000
pl_id                       0.000000
pl_length                   0.000000
pl_name                     0.000000
track_added                 0.425517
track_album_id              0.617978
track_album_name            0.267382
track_artists_id1           0.617978
track_artists_id2          75.638992
track_artists_id3          92.742319
track_artists_id4          97.734462
track_artists_id5          99.104549
track_artists_name1         0.195443
track_artists_name2        75.665253
track_artists_name3        92.749428
track_artists_name4        97.737634
track_artists_name5        99.104663
track_explicit              0.000000
track_href                  0.617978
track_id                    0.617978
track_isrc                  0.760786
track_length_ms             0.000000
track_name                  0.070754
track_populatrity           0.000000
track_acousticness          0.745305
t

### Save the final dataframe as a pickle file for use in other notebooks.

In [242]:
# spotify_data.to_pickle(data_folder + "spotify_data")